In [1]:
# setting
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()

# jupyter notebook 上で非同期コードを実行するために必要
nest_asyncio.apply()

### **Agents**
https://ai.pydantic.dev/agents/#introduction

In [ ]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(  
    'openai:gpt-4o',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:  
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'


# Run the agent
success_number = 18  
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(result)
print(result.data)  
#> True

result = roulette_agent.run_sync('I bet five is the winner', deps=success_number)
print(result)
print(result.data)
#> False

In [ ]:
print(result.cost())

### **Results**
https://ai.pydantic.dev/results/#result-validators-functions

In [ ]:
from typing import Union

from pydantic import BaseModel
from pydantic_ai import Agent, RunContext, ModelRetry

class QueryError(Exception):
    pass

class DatabaseConn:
    async def execute(self, query: str) -> None:
        if 'DROP' in query:
            raise QueryError('DROP is not allowed')
        print(f'Executing: {query}')

class Success(BaseModel):
    sql_query: str

class InvalidRequest(BaseModel):
    error_message: str

Response = Union[Success, InvalidRequest]
agent: Agent[DatabaseConn, Response] = Agent(
    'gemini-1.5-flash',
    result_type=Response,  # type: ignore
    deps_type=DatabaseConn,
    system_prompt='ユーザー入力に基づいてPostgreSQL風のSQLクエリを生成してください。',
)

@agent.result_validator
async def validate_result(ctx: RunContext[DatabaseConn], result: Response) -> Response:
    if isinstance(result, InvalidRequest):
        return result
    try:
        await ctx.deps.execute(f'EXPLAIN {result.sql_query}')
    except QueryError as e:
        raise ModelRetry(f'Invalid query: {e}') from e
    finally:
        return result

result = agent.run_sync(
    '昨日アクティブだったユーザーを取得してください。', deps=DatabaseConn()
)
print(result.data)
#> sql_query='SELECT * FROM users WHERE last_active::date = today() - interval 1 day'

In [ ]:
print(result.all_messages())

### **System Prompts**
https://ai.pydantic.dev/agents/#system-prompts

In [ ]:
from datetime import date

from pydantic_ai import Agent, RunContext

agent = Agent(
    'openai:gpt-4o',
    deps_type=str,
    system_prompt="顧客の名前を使って返信してください。",
)

@agent.system_prompt
def add_the_users_name(ctx: RunContext[str]) -> str:
    return f"ユーザーの名前は {ctx.deps} です。"

@agent.system_prompt
def add_the_date() -> str:
    return f'今日の日付は {date.today()} です。'

result = agent.run_sync('今日の日付は何ですか？', deps='Frank')
print(result.data)
#> 今日の日付は 2024-12-09 です。

In [ ]:
import pprint

pprint.pprint(result.all_messages())

### **Tools**
https://ai.pydantic.dev/tools/

In [ ]:
import random

from pydantic_ai import Agent, RunContext

agent = Agent(
    'gemini-1.5-flash',
    deps_type=str,
    system_prompt=(
        "あなたはサイコロゲームです。サイコロを振って出た数字がユーザーの予想と一致するか確認してください。"
        "一致した場合は、ユーザーに勝者であることを伝えてください。"
        "返答にはプレイヤーの名前を使用してください。"
    ),
)

@agent.tool_plain
def roll_die() -> str:
    """サイコロを振って出た数字を返してください。"""
    return str(random.randint(1, 6))

@agent.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """ユーザーの名前を返してください。"""
    return ctx.deps

dice_result = agent.run_sync('私の予想は4です', deps='Anne')
print(dice_result.data)
#> サイコロを振って出た数字は 4 です。
#> あなたは勝者です！

In [ ]:
import pprint

pprint.pprint(dice_result.all_messages())

### **Multi-Turn**

In [ ]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o')

messages = []

for _ in range(10):
    user_input = input("USER: ")
    if user_input.lower() == 'quit':
        print("チャットを終了します。さようなら。")
        break

    result = agent.run_sync(user_input, message_history=messages)
    print("ASSISTANT:", result.data)
    
    messages = result.all_messages()